In [ ]:
!git clone https://github.com/1stDayHack/FDK.git
!pip install -r FDK/requirements.txt
!pip install matplotlib==3.1.3

In [ ]:
import torch
torch.__version__

In [ ]:
!python -m pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cp drive/MyDrive/A_zz_random_stuffs/Books/1stdayhackReadings/RRDB_ESRGAN_x4.pth FDK/src/core/base_libs/ESRGAN/models/RRDB_ESRGAN_x4.pth

In [ ]:
#Import libs
from FDK.src.core.detect import Detector
from FDK.src.core.super_res import SuperReser
from FDK.src.core.utils import utils

from PIL import Image

import os
import matplotlib.pyplot as plt
import numpy

In [ ]:
det = Detector(model="COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")

In [ ]:
from pprint import pprint
pprint(det.cfg)

In [ ]:
img = Image.open("kisedje.jpeg")
img_cv = utils.pil_to_cv2(img)

In [ ]:
output = det.predict(img_cv)
out_img = det.visualize(img_cv, output, figsize=(18,18))

In [ ]:
pprint(det.dataset_metadata)

# 5.5 Tutorial Section

1. Use `Detector` from the 1stDayKit or FDK module of `detect` to perform the task of object-detection, object-segmentation and object-classification on a test image. For example, you can try it out on this: [`https://mymodernmet.com/wp/wp-content/uploads/2018/01/Tatsuto-Shibata-tokyo-photography-13.jpg`](https://mymodernmet.com/wp/wp-content/uploads/2018/01/Tatsuto-Shibata-tokyo-photography-13.jpg) 

***Hint***: If you are working on Colab, a quick way to download images using image urls directly to your Colab working directory, you can use `wget` to do so. For example try to run `!wget [https://mymodernmet.com/wp/wp-content/uploads/2018/01/Tatsuto-Shibata-tokyo-photography-13.jpg](https://mymodernmet.com/wp/wp-content/uploads/2018/01/Tatsuto-Shibata-tokyo-photography-13.jpg)` in a cell and look refresh your current Colab directory.

In [ ]:
!wget https://mymodernmet.com/wp/wp-content/uploads/2018/01/Tatsuto-Shibata-tokyo-photography-13.jpg

In [ ]:
objDet = Detector(model="COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml", device='cuda')

In [ ]:
img = Image.open("Tatsuto-Shibata-tokyo-photography-13.jpg")
img_cv = utils.pil_to_cv2(img)

In [ ]:
output = objDet.predict(img_cv)
out_img = objDet.visualize(img_cv, output, figsize=(18,18))

In [ ]:
objSegDet = Detector(model="COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

In [ ]:
output = objSegDet.predict(img_cv)
out_img = objSegDet.visualize(img_cv, output, figsize=(18,18))

2. Now, try to modify the `Detector` from above and perform pose-estimation on the same image instead. An example of pose-estimation is as shown in the image below. Also, there is more information on what pose-estimation is in a link given in the Further Readings article for this chapter.

***Hint***: *Check out Lesson 2 of this chapter if you have not!*

In [ ]:
poseDet = Detector(model="COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml")

In [ ]:
!mkdir processed

In [ ]:
output = poseDet.predict(img_cv)
out_img = poseDet.visualize(img_cv, output, figsize=(18,18), noplot=True)

In [ ]:
poseDet.save_image(out_img, 'processed', file_name='japanPosers')

3. Now the harder challenge yet, try to perform the same task as `Q2` but this time do so with a video (e.g. image sequence)! Use the video located at [`https://drive.google.com/file/d/1Wv_-IUQtNFEs5xv3wFEerwymQXvZ_gsR/view?usp=sharing`](https://drive.google.com/file/d/1Wv_-IUQtNFEs5xv3wFEerwymQXvZ_gsR/view?usp=sharing), a beautiful little 20 second sequence that I have found online. Your task would be to apply the model (e.g perform model inference) on all the frames in the sequence.

[Input Description]
A video sequence (from above) in the form of jpeg images.

[Output Description] 
The same jpeg images, but this time, with all the boxes and labels from the model drawn on them. You do not need to save the these images (e.g. output from the model) out as actual jpegs in your local directory. Just visualizing the images in your Notebook cell is sufficient.

***Hint***: *First try to convert the video into a sequence of jpeg images. You can convert video files such as mp4 into jpeg images very simply through existing online tools - try googling for it! Alternatively, you can always use other 3rd party tools to do so, such as `ffmpeg` if you're familiar with it.*

In [ ]:
%cp drive/MyDrive/A_zz_random_stuffs/Books/1stdayhackReadings/nyc_native_3_4.mp4 nyc_native_3_4.mp4

In [ ]:
!rm -rf nyc_vid_frames
!rm -rf processed_nyc_vid_frames

In [ ]:
!mkdir nyc_vid_frames

In [ ]:
!mkdir processed_nyc_vid_frames

In [ ]:
finalFPS = 20

In [ ]:
objdet = Detector(model="COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml", device='cuda')

In [ ]:
import cv2
vidcap = cv2.VideoCapture('nyc_native_3_4.mp4')
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite("nyc_vid_frames/frame"+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames
sec = 0
frameRate = 1/finalFPS #capture frame every <1/finalFPS> second
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

In [ ]:
import os
frames = os.listdir('nyc_vid_frames')

for i in range(1, len(frames)+1):
  frame = Image.open('nyc_vid_frames/frame'+ str(i) +'.jpg')
  frame_cv = utils.pil_to_cv2(frame)

  frame_output = objdet.predict(frame_cv)
  frame_output_img = objdet.visualize(frame_cv, frame_output, figsize=(18,18), noplot=True)

  filename = 'p_frame'+str(i)

  objdet.save_image(frame_output_img, 'processed_nyc_vid_frames', file_name=filename)

  # so we know until where it going
  print('processed '+str(i)+'th frame')

In [ ]:
import cv2
import numpy as np
import os
from os.path import isfile, join

pathIn= './processed_nyc_vid_frames/'
pathOut = './processed/video.avi'
fps = finalFPS
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

#for sorting the file names properly
files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
print(files)

for i in range(len(files)):
    filename=pathIn + files[i]
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

4. [Optional, no answer provided] You could go the full mile and try to recreate what I did with the [demo](https://github.com/1stDayHack/FDK/raw/master/misc/readme/nyc_final_1.gif) on the 1stDayKit repo main page by saving all the resulting output images, and then converting said images back into a mp4 video for bonus awesome points 🌟. Try to create a video whereby the first half of it (e.g. half the frames) is raw (e.g. with no model output applied) and the second half consisting of inferred frames (e.g. with model output applied). In addition, use the `PIL` to put a text in a corner of your video frame to indicate whether it is displaying the `raw` images or the `output` images. 

***Hint***: *You can refer to this* [`https://github.com/1stDayHack/FDK/blob/master/demo_1.ipynb`](https://github.com/1stDayHack/FDK/blob/master/demo_1.ipynb) for more hints!

In [ ]:
# this code is left as a trivial practice for the code-examiner